In [ ]:
import pandas as pd

# self-writen stuff
from predict_data import prediction_data
from random_forest import predictor

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
path = '../better_con_score.csv'
combined = pd.read_csv(path, engine='c', compression='zip')
combined = combined[combined['adelay0'] != 0.0]

In [ ]:
combined.columns.values

In [ ]:
dataset = combined[['transtime', 'adelay0', 'adelay5', 'adelay10', 'adelay15', 'ddelay0', 'ddelay5', 'ddelay10', 'ddelay15', 'doable3']].to_numpy()

In [ ]:
del combined

In [ ]:
def calc_con_score(transfertime, pred1, pred2):
    con_score = np.zeros(len(pred1))
    print(len(pred1))
    if (transfertime > 17):
        con_score = 1 #if the transfer time is higher than our highest lable, we can only predict the connection as working

    elif (transfertime > 12):
        p1 = pred1[:, 3] * (1 - pred2[:, 3])
        con_score = 1 - (p1)

    elif (transfertime > 7):
        #if the arrival train has 10 min delay, and the departure one does not have 5 min delay
        p1 = (pred1[:, 2] - pred1[:, 3]) * (1 - pred2[:, 1])

        #if the arrival train has 15 min delay, and the departure one does not have 10 min delay
        p2 = pred1[:, 3] * (1 - pred2[:, 2])
        con_score = 1 - (p1+p2)

    else:
        p1 = (pred1[:, 1] - pred1[:, 2]) * (1 - pred2[:, 1])
        p2 = (pred1[:, 2] - pred1[:, 3]) * (1 - pred2[:, 2])
        p3 = pred1[:, 3] * (1 - pred2[:, 3])
        con_score = 1 - (p1+p2+p3)
    
    return con_score

In [ ]:
def train_and_test(X, y, X_test, y_test, name='better_con_score'):
    clf = ExtraTreesClassifier(n_estimators=20, max_depth=5,
                               random_state=0, n_jobs = 4, criterion = 'entropy')
    # LogisticRegression(random_state=0)
    clf.fit(X.astype('float'), y.astype('bool'))

    #make and print some acc statistics
    test(X_test, y_test, clf)


In [ ]:
def test(X_test, y_test, clf):
    test_pred = clf.predict_proba(X_test)
    evaluate_test(test_pred[:, 0], y_test)

In [ ]:
def evaluate_test(test_pred, y_test):
    bool_test_pred = test_pred > 0.5
    print(bool_test_pred)
    print(y_test)
    print(bool_test_pred == y_test)
    
    results = {}
    results['acc'] = len(test_pred[bool_test_pred == y_test]) / len(test_pred)
    for i in range(10):
        trues = len(test_pred[(test_pred > (i/10)) & (test_pred < (i/10+0.1)) & (bool_test_pred == y_test)])
        complete = len(test_pred[(test_pred > (i/10)) & (test_pred < (i/10+0.1))])
        print(trues, '/', complete)
        try:
            results['acc'+ str(i)] = trues / complete
        except ZeroDivisionError:
            results['acc'+ str(i)] = np.nan
    print(results)


In [ ]:
def np_sets(dataset):
    np.random.shuffle(dataset)
    data_train, data_test, labels_train, labels_test = train_test_split(dataset[:, :9], dataset[:, -1], test_size=0.10, random_state=42)

    return data_train, data_test, labels_train, labels_test

In [ ]:
X, x, Y, y = np_sets(dataset)

In [ ]:
con_score = calc_con_score(15, dataset[(dataset[:, 0] > 12).astype('bool') & (dataset[:, 0] < 18).astype('bool'), 1:5], dataset[(dataset[:, 0] > 12).astype('bool') & (dataset[:, 0] < 18).astype('bool'), 5:9])
dataset[(dataset[:, 0] > 12).astype('bool')  & (dataset[:, 0] < 18).astype('bool') , -1] = con_score
# dataset[dataset[:, 0] > 17, -1] = 1 

In [ ]:
y_t = dataset[(dataset[:, 0] > 12).astype('bool') & (dataset[:, 0] < 18).astype('bool'), -1] == True
evaluate_test(con_score, y_t)

In [ ]:
train_and_test(X, Y, x, y)

In [ ]:
len(combined[combined['doable3'] == True]) / len(combined)

In [ ]:
b = np.zeros((dataset.shape[0],dataset.shape[1]+1))
b[:,:-1] = dataset
dataset = b

In [ ]:
acc = dataset[(dataset[:, -1] > 0.5).astype('bool') == dataset[:, -2]]
# print('Feature importances', clf.feature_importances_)
print('acc_' + 'name' + ':', len(acc)/ len(dataset))